In [ ]:
###################################################
# Profile Mask Visualizer
###################################################

from cgmbrush.cgmbrush import *
import matplotlib.colors as colors




In [ ]:
profile = FireProfile() # Choose a profile

# pick some parameters
mass = 1E13 # solar masses
redshift = 0 
resolution = 8
cellsize = L/(1024 * resolution)
print("cellsize (Mpc): %s" % cellsize)
fine_mask_len = 20 * resolution
comoving_radius = (1+redshift)*(mass**(1/3) / Rvir_den(redshift)) # what are the units...
print(q(0))
print("comoving radius: %s" % comoving_radius)

mask = profile.get_mask(mass, comoving_radius, redshift, resolution, 1, cellsize, fine_mask_len)
fig, ax = plt.subplots(1,1,figsize=(12, 12))
pos = ax.imshow(mask)#, norm=colors.LogNorm(vmin=1E-1, vmax=mask.max()))
ax.title.set_text('Fine Mask, M=${:.1e}$'.format(mass))
fig.colorbar(pos, ax=ax)

In [ ]:
from cgmbrush import *
def compareRadii(mass, z):
    print("Mass {:.1e}, redshift {}".format(mass, z))
    a = comoving_radius_for_halo(mass, z)
    b = r200Mz(mass, z)
    print(' rVir:\t{:.4f}'.format(a))
    print(' r200:\t{:.4f}'.format(b))
    print(' difference:\t{:.1f}%'.format((abs(b-a)) * 100 / b))

masses = [1E10,1E13,1E15]
redshifts = [0,0.5,1,2]

for z in redshifts:
    for m in masses:
        compareRadii(m, z)

In [ ]:
p = BolshoiProvider()
old = Configuration(FireProfile(), 1, provider = p, resolution=8)
old.datestamp = '2021-11-01' 
#old.datestamp = '2021-11-16' # re ran it the old way today

new = Configuration(FireProfile(), 1, provider = p, resolution=8)
new.datestamp = '2021-11-08'

vmin = 0
vmax = 500
fig, axs = fields_comparison_plot([('FIRE - Calculate Interp', NEW), ('FIRE - Old Read Interp', OLD)], vmin, vmax)
